In [2]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from malnis import show

In [3]:
files = [x for x in os.listdir("quotes") if "json" in x]
files

['quotes_results_brendan.json',
 'quotes_results_naitik.json',
 'quotes_results_jaber.json',
 'quotes_results_aswini.json',
 'quotes_results_neha.json',
 'quotes_results_jyothis.json',
 'quotes_results_harshit.json',
 'quotes_results_monzurul.json']

In [4]:
dicts = []
for f in tqdm(files):
    with open("quotes/" + f, "r") as file:
        dicts.append(json.loads(file.read()))

  0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
raw = pd.DataFrame(dicts, columns = list(dicts[0].keys()))
show(raw)

(8, 3)


,sentences,query,history
0,"{""filename"":{""0"":""Lin et al. - 2019 - Using So...",Hydrothermal vents are unique deep-sea structu...,"{""filename"":{""96"":""Lin et al. - 2019 - Using S..."
1,"{""filename"":{""0"":""Paper1.pdf"",""1"":""Paper1.pdf""...",Cloud computing provides a shared pool of comp...,"{""filename"":{""283"":""Paper1.pdf"",""3"":""Paper1.pd..."
2,"{""filename"":{""0"":""two level boolean rules.pdf""...",Prognostic modelling using machine learning te...,"{""filename"":{""847"":""ROPE (robust MUSE).pdf"",""2..."
3,"{""filename"":{""0"":""obesity.pdf"",""1"":""obesity.pd...",Obesity is a significant problem in population...,"{""filename"":{""1013"":""fnut-08-669155.pdf"",""540""..."
4,"{""filename"":{""0"":""Verification Based Authentic...",A Scenario-based Trust management\nApproach wi...,"{""filename"":{""409"":""VANET Security Framework f..."


In [6]:
dfs = raw[["history", "query"]]\
.assign(history = lambda df: df.history.map(pd.read_json))\
.assign(
    history = lambda df: df.apply(
        lambda r: r["history"].assign(query = r["query"])\
        .sort_values(["filename", "sentence"]).groupby("filename").agg(list).reset_index(),
        axis = 1
    )
)

show(dfs)

(8, 2)


,history,query
0,fi...,Hydrothermal vents are unique deep-sea structu...
1,filename ...,Cloud computing provides a shared pool of comp...
2,filename \ 0 ...,Prognostic modelling using machine learning te...
3,filename \ 0 ...,Obesity is a significant problem in population...
4,fi...,A Scenario-based Trust management\nApproach wi...


In [12]:
data = pd.concat(dfs.history.tolist())\
.assign(
    query = lambda df: df["query"].map(min),
    relevance = lambda df: df.relevance.map(lambda x: [1 if y in (1, True) else 0 for y in x])
)\
[["text", "query", "relevance"]]\
.reset_index(drop = True)

show(data, 30)

(28, 3)


,text,query,relevance
0,"[29, R425–R473 Forum Using Soundscapes to Asse...",Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
1,"[Therefore, using daily SEL to describe sounds...",Hydrothermal vents are unique deep-sea structu...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
2,[TECHNOLOGY AND CODE published: 19 August 2021...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,[ORIGINAL RESEARCH published: 05 August 2021 d...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ..."
4,[A brief overview of current approaches for un...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, ..."
5,[Due to its tremendous advantages this technol...,Cloud computing provides a shared pool of comp...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]"
6,[Interpretable & Explorable Approximations of ...,Prognostic modelling using machine learning te...,"[1, 0, 1, 0, 1, 0]"
7,"[Introduction Over the past decade, there has ...",Prognostic modelling using machine learning te...,"[1, 1, 1, 0, 1, 1, 0, 0]"
8,[Model-agnostic interpretability of machine le...,Prognostic modelling using machine learning te...,"[0, 0]"
9,"[To gain insight into the black-box model, we ...",Prognostic modelling using machine learning te...,"[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]"


In [8]:
sum([pd.read_json(d["history"]).shape[0] for d in dicts])

335

In [9]:
data.text.map(len).sum()

335

In [23]:
f = data.relevance.map(lambda x: all([0 if y == 1 else 1 for y in x]) or all(x))
f

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10     True
11    False
12    False
13     True
14     True
15     True
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
Name: relevance, dtype: bool

In [26]:
data.loc[~f].to_pickle("quotes.pkl")